In [40]:
import os
import numpy as np
import laspy
import glob
import os.path as osp
import itertools
import multiprocessing as mp
from tqdm import tqdm

In [47]:
def read_file(file_path):
    las = laspy.read(file_path)
    xyz = las.xyz
    labels = las.classification
    pc = np.column_stack([xyz, labels])
    return pc

In [65]:
def slice_pts(pc, bx, by, bz, box_dim):
    pc_slice = pc[(pc[:, 0] >= bx) & (pc[:, 0] <= bx + box_dim) &
                  (pc[:, 1] >= by) & (pc[:, 1] <= by + box_dim) &
                  (pc[:, 2] >= bz) & (pc[:, 2] <= bz + box_dim)]
    
    if len(pc_slice) > 2000:
        if len(pc_slice) > 20000:
            indices = np.random.choice(pc_slice.shape[0], size=20000, replace=False)
            pc_slice = pc_slice[indices]
        
        return pc_slice
    
def process_blocks(params):
    pc, bx, by, bz, box_dim = params
    return slice_pts(pc, bx, by, bz, box_dim)
            

In [66]:
def preprocess_point_cloud(pc, box_dim=6, box_overlap=0.5):
    xmin, ymin, zmin = np.floor(np.min(pc[:, :3], axis=0))
    xmax, ymax, zmax = np.ceil(np.max(pc[:, :3], axis=0))
    
    box_overlap = box_dim * box_overlap
    
    x_cnr = np.arange(xmin - box_overlap, xmax + box_overlap, box_overlap)
    y_cnr = np.arange(ymin - box_overlap, ymax + box_overlap, box_overlap)
    z_cnr = np.arange(zmin - box_overlap, zmax + box_overlap, box_overlap)
    
    slices_blocks = []
    params_list = [(pc, bx, by, bz, box_dim) for bx, by, bz in itertools.product(x_cnr, y_cnr, z_cnr)]
    
    with mp.Pool(processes=23) as pool:
        for result in tqdm(pool.imap_unordered(process_blocks, params_list), total=len(params_list)):
            if result is not None:
                slices_blocks.append(result)
    return slices_blocks

In [67]:
ROOT_DIR = "../data/NIBIO_MLS"
RAW_DIR = osp.join(ROOT_DIR, "raw")


files = glob.glob(osp.join(RAW_DIR, "train/*.las"), recursive=True)

sliced_blocks = []
for file in files:
    pc = read_file(file)
    sliced_block = preprocess_point_cloud(pc)
    sliced_blocks += sliced_block

100%|██████████| 637/637 [00:08<00:00, 78.72it/s]


In [68]:
len(sliced_blocks)

4814

In [71]:
sorted_blocks = sorted(sliced_blocks, key=lambda x : x.shape[0], reverse=True)
sorted_blocks

[array([[-73.068,  -2.592,   3.486,   0.   ],
        [-74.306,  -4.009,   2.119,   0.   ],
        [-73.013,  -6.033,   1.875,   0.   ],
        ...,
        [-72.222,  -5.307,   1.271,   0.   ],
        [-69.264,  -2.809,   0.775,   0.   ],
        [-72.83 ,  -3.752,   2.557,   0.   ]], shape=(20000, 4)),
 array([[-73.108,  -3.283,  -5.442,   0.   ],
        [-71.266,  -1.953,  -3.817,   0.   ],
        [-73.665,  -1.193,  -6.142,   0.   ],
        ...,
        [-69.42 ,  -2.829,  -3.685,   0.   ],
        [-71.315,  -1.965,  -1.086,   0.   ],
        [-74.786,  -1.295,  -2.966,   0.   ]], shape=(20000, 4)),
 array([[-64.131,  -1.316,  -3.796,   0.   ],
        [-64.427,  -1.88 ,  -4.545,   0.   ],
        [-63.091,  -1.841,  -2.557,   0.   ],
        ...,
        [-64.46 ,  -1.15 ,  -4.343,   0.   ],
        [-63.402,  -1.872,  -2.448,   0.   ],
        [-63.892,  -1.429,  -5.914,   0.   ]], shape=(20000, 4)),
 array([[-63.622,  -2.331,  -4.865,   0.   ],
        [-64.615,  -0.125, 

In [72]:
import pickle

with open(osp.join(ROOT_DIR, "processed", "nibio_mls_6m_2000-20000points_train.pkl"), "wb") as f:
    pickle.dumps(sliced_blocks)